In [28]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [29]:
# Load the cleaned accident data
df = pd.read_csv('D:/accident_severity_analytics/data/processed/Cleaning_Crashes_Data.csv',index_col=0)

In [30]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1('Accident Analysis Dashboard', style={'textAlign': 'center', 'margin': '20px'}),

    # Refresh button
    html.Button('Refresh Data', id='refresh-button', style={'margin': '10px'}),

    # Sidebar for filters
    html.Div([
        html.H3('Filters', style={'textAlign': 'left'}),
        dcc.Dropdown(
            id='vehicle-type',
            options=[{'label': v, 'value': v} for v in df['vehicle type code 1'].unique()],
            placeholder='Select Vehicle Type'
        ),
        dcc.DatePickerRange(
            id='date-range',
            start_date=df['crash date'].min(),
            end_date=df['crash date'].max(),
            display_format='YYYY-MM-DD'
        ),
    ], style={'width': '20%', 'float': 'left', 'padding': '20px', 'backgroundColor': '#f0f0f0'}),

    # Main content
    html.Div([
        html.H2('Overview'),
        html.P(f'Total Accidents: {len(df)}'),
        html.P(f'Total Injuries: {df["number of persons injured"].sum()}'),
        html.P(f'Total Fatalities: {df["number of persons killed"].sum()}'),

        # Time Distribution Analysis
        html.H2('Time Distribution of Accidents'),
        dcc.Graph(id='time-distribution'),

        # High-Risk Locations Analysis
        html.H2('High-Risk Area Clustering'),
        dcc.Graph(id='cluster-map'),

        # Injuries and Fatalities Correlation
        html.H2('Injuries vs Fatalities'),
        dcc.Graph(id='injuries-vs-fatalities'),

        # Most Common Causes of Accidents
        html.H2('Most Common Accident Causes'),
        dcc.Graph(id='common-causes'),
    ], style={'width': '75%', 'float': 'right', 'padding': '20px'})
])

# Callbacks for data refresh
@app.callback(
    dash.dependencies.Output('total-accidents', 'children'),
    dash.dependencies.Output('total-injuries', 'children'),
    dash.dependencies.Output('total-fatalities', 'children'),
    dash.dependencies.Input('refresh-button', 'n_clicks')
)
def update_overview(n_clicks):
    refresh_data()
    return (
        f'Total Accidents: {len(df)}',
        f'Total Injuries: {df["number of persons injured"].sum()}',
        f'Total Fatalities: {df["number of persons killed"].sum()}'
    )

# Callbacks for interactivity
@app.callback(
    dash.dependencies.Output('time-distribution', 'figure'),
    [dash.dependencies.Input('vehicle-type', 'value'),
     dash.dependencies.Input('date-range', 'start_date'),
     dash.dependencies.Input('date-range', 'end_date')]
)
def update_time_distribution(vehicle_type, start_date, end_date):
    filtered_df = df[(df['crash date'] >= start_date) & (df['crash date'] <= end_date)]
    if vehicle_type:
        filtered_df = filtered_df[filtered_df['vehicle type code 1'] == vehicle_type]
    fig = px.histogram(filtered_df, x='crash date', title='Accident Count Over Time')
    return fig

@app.callback(
    dash.dependencies.Output('cluster-map', 'figure'),
    [dash.dependencies.Input('vehicle-type', 'value')]
)
def update_cluster_map(vehicle_type):
    filtered_df = df
    if vehicle_type:
        filtered_df = df[df['vehicle type code 1'] == vehicle_type]
    fig = px.scatter_map(filtered_df, lat='latitude',
                     lon='longitude', color='number of persons injured',
                     size='number of persons injured',
                     title='High-Risk Area Clustering', zoom=10)

    return fig

@app.callback(
    dash.dependencies.Output('injuries-vs-fatalities', 'figure'),
    [dash.dependencies.Input('vehicle-type', 'value')]
)
def update_injuries_vs_fatalities(vehicle_type):
    filtered_df = df
    if vehicle_type:
        filtered_df = df[df['vehicle type code 1'] == vehicle_type]
    fig = px.scatter(filtered_df, x='number of persons injured', y='number of persons killed', color='borough', title='Injuries vs Fatalities')
    return fig

@app.callback(
    dash.dependencies.Output('common-causes', 'figure'),
    [dash.dependencies.Input('vehicle-type', 'value')]
)
def update_common_causes(vehicle_type):
    filtered_df = df
    if vehicle_type:
        filtered_df = df[df['vehicle type code 1'] == vehicle_type]
    cause_counts = filtered_df['contributing factor vehicle 1'].value_counts().head(10)
    fig = px.bar(x=cause_counts.index, y=cause_counts.values, labels={'x': 'Cause', 'y': 'Count'}, title='Most Common Accident Causes')
    return fig

app.run(jupyter_mode='external')


Dash app running on http://127.0.0.1:8050/
